### Games data cleanup

This notebook cleans up the games.json file in preparation for use in a Pandas dataframe.

In [ ]:
import json
import pandas as pd
from datetime import datetime
from dateutil import parser

In [ ]:
df = pd.read_json('raw_stats.json')

This cell drops annulled games and some columns that I considered irrelevant for my personal analysis. You should customize this to your own purposes - for instance, I do not play tournaments, so I dropped these columns.

In [ ]:
# Drop annulled games, then drop annulled column and re-index
df.drop(df.index[df['annulled'] == True].tolist(), inplace = True)
df.drop(['annulled'], axis = 1, inplace = True)
df.reset_index(drop = True, inplace = True)

# Convert [width, height] information to single column (assuming you only play square board sizes)
df.drop(['width'], axis = 1, inplace = True)
df = df.rename(columns = {"height":"size"})

# Define irrelevant columns - customize this according to individual need
deprecated = ['black_player_rating', 'white_player_rating', 'black_player_rank', 'white_player_rank'] # these are from before the pre-2017 Glicko adoption
rengo = ['rengo', 'rengo_black_team', 'rengo_white_team', 'rengo_casual_mode'] # keep these if you want to keep rengo info
game_metadata = ['players', 'name', 'sgf_filename', 'flags', 'creator', 'source', 'mode']
tournament = ['tournament', 'tournament_round', 'ladder'] # keep these if you want to keep tournament info
correspondence = ['pause_on_weekends'] # keep this if you play correspondence with this setting and think it's relevant

# Drop above columns
df.drop(deprecated, axis = 1, inplace = True)
df.drop(rengo, axis = 1, inplace = True)
df.drop(game_metadata, axis = 1, inplace = True)
df.drop(tournament, axis = 1, inplace = True)
df.drop(correspondence, axis = 1, inplace = True)

This cell reorganizes the dataframe in terms of a me-vs-opponent perspective rather than a black-vs-white perspective. For instance, instead of `{"black":"my_id", "white":"opponent_id"}`, we have `{"my_color":"black"}`. This is applied to:
* player colors
* player IDs
* game results

In [ ]:
myID =  511491
playedBlack = df.index[df['black'] == myID].tolist()
playedWhite = df.index[df['white'] == myID].tolist()

my_color = ['white'] * df.shape[0]
opponent_id = df.get('black')
won = [False] * df.shape[0]

df.insert(3, 'my_color', my_color)
df.insert(4, 'opponent_id', won)
df.insert(5, 'won', won)
df.loc[playedBlack, 'my_color'] = 'black'
df.loc[playedBlack, 'opponent_id'] = df.loc[playedBlack, 'white']
df.loc[playedWhite, 'opponent_id'] = df.loc[playedWhite, 'black']
df.drop(['black', 'white'], axis = 1, inplace = True)

df.loc[playedBlack, 'won'] = df.loc[playedBlack, 'white_lost'] 
df.loc[playedWhite, 'won'] = df.loc[playedWhite, 'black_lost']
df.drop(['black_lost', 'white_lost'], axis = 1, inplace = True)

This cell extracts the live/blitz/correspondence information from the `'time_control_parameters'` column. There is other potentially relevant information in here but I have chosen to drop it.

In [ ]:
speed = [json.loads(df['time_control_parameters'][i])['speed'] for i in range(df.shape[0])]
df.insert(9, 'speed', speed)
df.drop(['time_control_parameters'], axis = 1, inplace = True)

This cell computes the length of each game (to the minute, rounded down), then drops the `'ended'` column.

In [ ]:
print(df['started'][0])
parser.parse(df['started'][0])
game_length = [divmod((parser.parse(df['ended'][i]) - parser.parse(df['started'][i])).seconds, 60)[0] for i in range(df.shape[0])]
df.insert(0, 'game_length', game_length)
df.drop(['ended'], axis = 1, inplace = True)
df.head()

This cell organizes the ratings data at the time of each game.

In [ ]:
# Extract historical ratings data and store in ['my_rating', 'my_deviation', 'my_volatility',
#                                               'opp_rating', 'opp_deviation', 'opp_volatility']
black_rating = pd.DataFrame([df['historical_ratings'][i]['black']['ratings']['overall']['rating'] 
                             for i in range(df.shape[0])], columns = ['black_rating'])
black_deviation = pd.DataFrame([df['historical_ratings'][i]['black']['ratings']['overall']['deviation'] 
                                for i in range(df.shape[0])], columns = ['black_deviation'])
black_volatility = pd.DataFrame([df['historical_ratings'][i]['black']['ratings']['overall']['volatility'] 
                                 for i in range(df.shape[0])], columns = ['black_volatility'])
white_rating = [df['historical_ratings'][i]['white']['ratings']['overall']['rating'] for i in range(df.shape[0])]
white_deviation = [df['historical_ratings'][i]['white']['ratings']['overall']['deviation'] for i in range(df.shape[0])]
white_volatility = [df['historical_ratings'][i]['white']['ratings']['overall']['volatility'] for i in range(df.shape[0])]

df.insert(0, 'my_rating', white_rating)
df.insert(0, 'opponent_rating', white_rating)
df.insert(0, 'my_deviation', white_deviation)
df.insert(0, 'opponent_deviation', white_deviation)
df.insert(0, 'my_volatility', white_volatility)
df.insert(0, 'opponent_volatility', white_volatility)

df.loc[playedBlack, 'my_rating'] = black_rating.loc[playedBlack, 'black_rating']
df.loc[playedWhite, 'opponent_rating'] = black_rating.loc[playedWhite, 'black_rating']
df.loc[playedBlack, 'my_deviation'] = black_deviation.loc[playedBlack, 'black_deviation']
df.loc[playedWhite, 'opponent_deviation'] = black_deviation.loc[playedWhite, 'black_deviation']
df.loc[playedBlack, 'my_volatility'] = black_volatility.loc[playedBlack, 'black_volatility']
df.loc[playedWhite, 'opponent_volatility'] = black_volatility.loc[playedWhite, 'black_volatility']

df.drop(['historical_ratings'], axis = 1, inplace = True)

This cell reorganizes the columns according to the following paradigm:

1. Game start time
2. Game id information
3. Player id information
4. Result information
5. Rules information
6. Time information
7. Player rank information

It then sorts the dataframe by game start time (descending order). (At this point the start times are converted from `str` to `datetime` objects.)

In [ ]:
cols = ['started',
        'id', 'ranked',
        'opponent_id', 'my_color',
        'won', 'outcome',
        'size', 'rules', 'handicap', 'komi', 'disable_analysis',
        'time_control', 'time_per_move', 'game_length',
        'my_rating', 'opponent_rating', 'my_deviation', 'opponent_deviation', 'my_volatility', 'opponent_volatility',
        'related']
df = df[cols]
df['started'] = pd.to_datetime(df['started']).values
df.sort_values(by = 'started', inplace = True, ascending = False)
df.reset_index(drop = True, inplace = True)

In [ ]:
df.head()

This cell stores our cleaned dataframe in a .json file for easy access during future analysis.

In [ ]:
df.to_json(r'clean_stats.json')